In [2]:
import faculty.datasets as datasets
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import spacy
import re
from utils import clean3, is_pua, remove_punct_nums
from sentence_transformers import SentenceTransformer, util
import pickle
from tqdm.notebook import tqdm
import torch
%matplotlib inline

2023-03-27 02:46:46.781915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda/envs/Python3/lib:
2023-03-27 02:46:46.781952: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-27 02:46:46.781971: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cube-783b66ca-1ff2-4661-bbfb-4788c0e9457a-6fc496d88c-8bt5r): /proc/driver/nvidia/version does not exist


In [3]:
word_df = pd.read_csv('../data/final/clean_12k_JNNkeyphrases_100_similar.csv')
cos_df = pd.read_csv('../data/final/clean_12k_JNNkeyphrases_100_similar_cos_sim.csv')

In [4]:
word_df = word_df.drop(['Unnamed: 0'],axis=1)

In [5]:
cos_df = cos_df.drop(['Unnamed: 0'],axis=1)

In [6]:
word_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,poor quality housing,poor housing conditions,housing quality,better quality housing,better housing,inefficient housing,housing need,housing improvement,housing improvements,housing needs,...,efficient homes,neighbourhoods,social housing parks,private housing finance,low income residents,low incomes,housing energy,low income households,figure housing,sustainable homes
1,figure david,sir david,plan figure,figure uk,figure fi,figures,figure moray,carbon figure,fig,footprint figure,...,greater manchester vision,programme,bike,planet,acts,progress figures,tonnes,major challenge,plots,bigger challenge
2,local governments,local government,local government structure,local government sector,local councils,local government body,local government association,local government funding,local government reorganisation,local government report,...,councils housing,community governance,public sector,uk councils,local,london councils,councils scope,citizens,local plan use,district council
3,transport council,council transport,public transport council,sustainable transport council,sustainable transport committee,infrastructure council,transport consultation,council travel,transport governance,transport services,...,domestic transport,sustainable transport infrastructure,environment council,sustainable transport solutions,edinburgh council,sustainable transport result,council operations,finance council,electrification council,transport system
4,increase tree,enhance trees,additional trees,more trees,additional tree planting,new tree management,further tree planting,extra trees,tree replacement,tree strategy,...,forestry carbon,increases ensure,deforestation,improve,further rise,more decrease,executive summary forestry,increase reuse,northern forest project,raising
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12820,direct debit system,direct operations transport,savings indirect,direct delivery,accounting,bill management,electronic billing,accounting standards,bank balance,digital post system,...,neutral programmes world,new financing mechanisms,neutral hull communications,control systems,savings annum,through provision,bill requirements,exchange,audits,energy system
12821,catering schools,school meals,secondary schools,schools housing,services school,food supply,local food production,food accounting,support schools,food production,...,various community facilities,local needs,childrens services,diet infrastructure,community centres,local dairy farm,farmers markets,teacher participation,invitations,learning environment
12822,uk net,national net,wider uk network,global net,council net,net uk carbon,newcastle net,net supporting,net support,uk,...,net balance,net sequestration,uk growth,westminster,net contributor,cornwall,connectivity,london environment,port glasgow,northern ireland
12823,estate changes,corporate estate changes,estate,estate action,estate operations,own estate,building estate,estates,estate management,own land estate,...,new dwellings,change management,transformational change,new housing development,change duties,housing improvement,council property,housing growth,transformational changes,homes sector


In [7]:
df1 = word_df
df2 = cos_df

In [8]:
df1

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,poor quality housing,poor housing conditions,housing quality,better quality housing,better housing,inefficient housing,housing need,housing improvement,housing improvements,housing needs,...,efficient homes,neighbourhoods,social housing parks,private housing finance,low income residents,low incomes,housing energy,low income households,figure housing,sustainable homes
1,figure david,sir david,plan figure,figure uk,figure fi,figures,figure moray,carbon figure,fig,footprint figure,...,greater manchester vision,programme,bike,planet,acts,progress figures,tonnes,major challenge,plots,bigger challenge
2,local governments,local government,local government structure,local government sector,local councils,local government body,local government association,local government funding,local government reorganisation,local government report,...,councils housing,community governance,public sector,uk councils,local,london councils,councils scope,citizens,local plan use,district council
3,transport council,council transport,public transport council,sustainable transport council,sustainable transport committee,infrastructure council,transport consultation,council travel,transport governance,transport services,...,domestic transport,sustainable transport infrastructure,environment council,sustainable transport solutions,edinburgh council,sustainable transport result,council operations,finance council,electrification council,transport system
4,increase tree,enhance trees,additional trees,more trees,additional tree planting,new tree management,further tree planting,extra trees,tree replacement,tree strategy,...,forestry carbon,increases ensure,deforestation,improve,further rise,more decrease,executive summary forestry,increase reuse,northern forest project,raising
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12820,direct debit system,direct operations transport,savings indirect,direct delivery,accounting,bill management,electronic billing,accounting standards,bank balance,digital post system,...,neutral programmes world,new financing mechanisms,neutral hull communications,control systems,savings annum,through provision,bill requirements,exchange,audits,energy system
12821,catering schools,school meals,secondary schools,schools housing,services school,food supply,local food production,food accounting,support schools,food production,...,various community facilities,local needs,childrens services,diet infrastructure,community centres,local dairy farm,farmers markets,teacher participation,invitations,learning environment
12822,uk net,national net,wider uk network,global net,council net,net uk carbon,newcastle net,net supporting,net support,uk,...,net balance,net sequestration,uk growth,westminster,net contributor,cornwall,connectivity,london environment,port glasgow,northern ireland
12823,estate changes,corporate estate changes,estate,estate action,estate operations,own estate,building estate,estates,estate management,own land estate,...,new dwellings,change management,transformational change,new housing development,change duties,housing improvement,council property,housing growth,transformational changes,homes sector


In [9]:
df2 = cos_df
df2.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
       '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72',
       '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84',
       '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96',
       '97', '98', '99'],
      dtype='object')

In [10]:
df1_melt = df1.melt(id_vars=['0'], value_vars=df1.columns[1:], var_name='nth similar', value_name='word_b')
df2_melt = df2.melt(id_vars=['0'], value_vars=df2.columns[1:], var_name='nth similar', value_name='cosine_similarity')

# # merge the two dataframes on the common column 'word_b'
# result = pd.merge(df1, df2, on='word_b')[['0_x', 'word_a', 'cosine_similarity']]
# result.columns = ['word_a', 'word_b', 'cosine_similarity']

In [11]:
df1_melt.head(20)

,0,nth similar,word_b
0,poor quality housing,1,poor housing conditions
1,figure david,1,sir david
2,local governments,1,local government
3,transport council,1,council transport
4,increase tree,1,enhance trees
5,important role,1,role
6,change actions,1,change action
7,devastate economies,1,resilient economy
8,clearer pipeline,1,pipeline
9,methods,1,new methods


In [12]:
df2_melt.head(20)

,0,nth similar,cosine_similarity
0,1.000000,1,0.895772
1,1.000000,1,0.714430
2,1.000000,1,0.965810
3,1.000000,1,0.940174
4,1.000000,1,0.782769
5,1.000000,1,0.790030
6,1.000000,1,0.958086
7,1.000000,1,0.631615
8,1.000000,1,0.823098
9,1.000000,1,0.797725


In [13]:
df1_melt['cosine similarity'] = df2_melt['cosine_similarity']
df1_melt

,0,nth similar,word_b,cosine similarity
0,poor quality housing,1,poor housing conditions,0.895772
1,figure david,1,sir david,0.714430
2,local governments,1,local government,0.965810
3,transport council,1,council transport,0.940174
4,increase tree,1,enhance trees,0.782769
...,...,...,...,...
1269670,direct debit system,99,energy system,0.289898
1269671,catering schools,99,learning environment,0.346913
1269672,uk net,99,northern ireland,0.416167
1269673,estate changes,99,homes sector,0.432327


In [16]:
n = len(df1)
new_index = [i + j for i in range(n) for j in range(0, len(df1_melt), n)]

In [17]:
df_final = df1_melt.iloc[new_index]
df_final

,0,nth similar,word_b,cosine similarity
0,poor quality housing,1,poor housing conditions,0.895772
12825,poor quality housing,2,housing quality,0.880248
25650,poor quality housing,3,better quality housing,0.865403
38475,poor quality housing,4,better housing,0.786690
51300,poor quality housing,5,inefficient housing,0.774039
...,...,...,...,...
1218374,tree orchard,95,planting,0.418389
1231199,tree orchard,96,farmers markets,0.416959
1244024,tree orchard,97,sustainable woodland management,0.414370
1256849,tree orchard,98,planting schemes,0.414161


In [18]:
# df_final = df_final.drop(['nth similar'],axis=1)
df_final = df_final.rename(columns= {'0':'word_a'})

In [19]:
df_final

,word_a,nth similar,word_b,cosine similarity
0,poor quality housing,1,poor housing conditions,0.895772
12825,poor quality housing,2,housing quality,0.880248
25650,poor quality housing,3,better quality housing,0.865403
38475,poor quality housing,4,better housing,0.786690
51300,poor quality housing,5,inefficient housing,0.774039
...,...,...,...,...
1218374,tree orchard,95,planting,0.418389
1231199,tree orchard,96,farmers markets,0.416959
1244024,tree orchard,97,sustainable woodland management,0.414370
1256849,tree orchard,98,planting schemes,0.414161


In [20]:
# !python -m spacy download en

In [21]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
doc = nlp('did displaying words')
print (" ".join([token.lemma_ for token in doc]))

do display word


In [21]:
from nltk.stem import PorterStemmer
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def has_common_word(row):
    ps = PorterStemmer()
    words1 = set([ps.stem(word) for word in row['word_a'].split()])
    words2 = set([ps.stem(word) for word in row['word_b'].split()])
    return len(words1 & words2) > 0

# create a new column using the custom function
df_final['has_common_word'] = df_final.apply(has_common_word, axis=1)

do display word


In [24]:
df_final[df_final['has_common_word']==False].sample(n=20)

,word_a,nth similar,word_b,cosine similarity,has_common_word
729989,new cycle hub,55,bike scheme,0.387565,False
778985,erosion work,59,wet regions wetter,0.378472,False
1318439,vehicles pool,99,comments car,0.401351,False
845690,key steering role,64,driver training,0.391866,False
400503,global warming methane,30,carbon emission,0.523162,False
1205203,housing,90,residential sector,0.615679,False
1134047,contact officer,85,management emergency,0.358465,False
1130188,greenhouse gases,85,emissions use,0.512458,False
1078980,different capacity,81,storage achievement,0.376214,False
1066398,interest,80,appropriate funding,0.439914,False


In [22]:
df_final.to_csv('../data/final/clean_12k-JNNkeyphrases-word_a-word_b.csv')

In [50]:
df_final.loc[(df_final['cosine similarity'] > 0.7) &(df_final['has_common_word']==False)].sample(20)

,word_a,nth similar,word_b,cosine similarity,has_common_word
8228,mile boundaries,1,mileage,0.712869,False
80431,expenditure proposals,6,additional budget allocation,0.710987,False
38885,significant advantages,3,various benefits,0.735764,False
4446,dec,1,december,0.761303,False
589149,strategic approach,44,comprehensive strategy,0.706270,False
113168,energy usage,9,consumption electricity,0.785882,False
35689,needs,3,requires,0.702046,False
21732,terminology,2,term definition,0.783448,False
103832,reduces,8,further reductions,0.739486,False
38300,other partners,3,more partnership,0.734911,False


In [49]:
df_final[(df_final['word_a'] == 'charging points') & (df_final['has_common_word'] ==False)]

,word_a,nth similar,word_b,cosine similarity,has_common_word
318647,charging points,24,electric vehicle charger,0.470100,False
332065,charging points,25,electrification,0.469005,False
345483,charging points,26,rapid chargers,0.468550,False
358901,charging points,27,different charger types,0.459721,False
452827,charging points,34,centre electricity,0.391372,False
...,...,...,...,...,...
1271325,charging points,95,reputation none,0.294366,False
1284743,charging points,96,electrical vehicles,0.294049,False
1298161,charging points,97,stationary sources electricity,0.293912,False
1311579,charging points,98,electricity prices,0.293742,False


In [116]:
ps = PorterStemmer()
ps.stem('convert')

'convert'